In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import ascii, fits
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import Angle
#import splat
%matplotlib inline

In [4]:
import sys
sys.path

['/Users/caganze/research/J1624/notebooks',
 '/Users/caganze/research/J1624/notebooks/PATH',
 '/Users/caganze/research/kastredux',
 '/opt/anaconda3/lib/python38.zip',
 '/opt/anaconda3/lib/python3.8',
 '/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/caganze/.local/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages/aeosa',
 '/opt/anaconda3/lib/python3.8/site-packages/sedkit-1.1.3-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/astroquery-0.4.2.dev0-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/pyvo-1.1-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/certifi-2020.12.5-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/var/folders/d4/6cttt_zx5fx_x9p161tb_fd00000gn/T/tmpepbxd_bm']

In [4]:
result = Vizier.query_object("sirius")

In [5]:
Vizier.query_object?

In [13]:
coord=SkyCoord(ra=246.0684208*u.deg, dec= -32.2037824*u.deg)
result = Vizier.query_region(coord, radius=Angle(2, "arcsec"), catalog='Gaia')

In [14]:
result

TableList with 5 tables:
	'0:I/324/igsl3' with 18 column(s) and 1 row(s) 
	'1:I/345/gaia2' with 32 column(s) and 1 row(s) 
	'2:I/350/gaiaedr3' with 39 column(s) and 1 row(s) 
	'3:I/352/gedr3dis' with 10 column(s) and 1 row(s) 
	'4:VI/145/attitude' with 18 column(s) and 1 row(s) 

In [16]:
result[2]['Gmag', 'BPmag', 'RPmag']

Gmag,BPmag,RPmag
mag,mag,mag
float64,float64,float64
12.991280,14.183434,11.010448


In [ ]:
DATA_FOLDER='/users/caganze/research/J1624/data/'
FIG_FOLDER='/users/caganze/research/J1624/figures/'

In [ ]:
Vizier.ROW_LIMIT = -1 
catalogs = Vizier.get_catalogs('J/AJ/157/231')

In [ ]:
#kiman=catalogs[0].to_pandas()
kiman_master=Table(fits.open(DATA_FOLDER+'/MLSDSSGaiaDR2.fits')[1].data)

In [ ]:
keys0=[k for k in kiman_master.columns if k not in  ['PSFMAG', 'PSFMAG_ERR', 'EXTINCTION', 'MAG_2MASS', 'MAG_ERR_2MASS']]
kiman=kiman_master[keys0].to_pandas()

In [ ]:
lateMs=pd.read_excel(DATA_FOLDER+'/UCD_lateM_dwarf_precision_RV_20pc_thin_disk_population.xlsx')

In [ ]:
cth=Table(fits.open(DATA_FOLDER+'/DR7_TOT_6_1.fits')[1].data)

In [ ]:
keys=[k for k in cth.columns if k not in  ['PSFMAG', 'PSFMAGERR', 'EXTINCTION']]

In [ ]:
theissen=cth[keys].to_pandas()

In [ ]:
merged=pd.merge(theissen, kiman,  how='left', on=['MJD', 'FIBER', 'PLATE'],  validate='one_to_one')

In [ ]:
len(merged), len(kiman), len(theissen)

In [ ]:
#merged.EWHA_x

In [ ]:
plt.plot(merged.RA_x, merged.RA_y)
plt.plot(merged.DEC_x, merged.DEC_y)

In [ ]:
#merged.to_hdf(DATA_FOLDER+'/merged_Mdwarfs.h5', key='merged')

In [ ]:
gcs=Vizier.get_catalogs('J/A+A/530/A138')

In [ ]:
#cross-match with merged

gcs[0]

In [ ]:

coo_gcs = SkyCoord(gcs[0]['RAJ2000'], gcs[0]['DEJ2000'], unit=(u.hourangle, u.deg), obstime="J2000")
coo_gaia = SkyCoord(merged.RA_x.values*u.deg, merged.DEC_x.values*u.deg, obstime='J2000')


In [ ]:
#coo_gcs.ra

In [ ]:
#SkyCoord?

In [ ]:
idx_sdss, d2d_sdss, d3d_sdss = coo_gaia.match_to_catalog_sky(coo_gcs)


In [ ]:
d2d_sdss.arcsec

In [ ]:
plt.hist(d2d_sdss.arcsec, histtype='step', range=(0,100))
plt.xlabel('separation [arcsec]')
plt.tight_layout()

In [ ]:
#cross-match with merged

plt.hist(gcs[0]['Teff'], range=(300))

In [ ]:
gcs[0].to_pandas().to_csv(DATA_FOLDER+'/gcs_catalog.csv')

In [ ]:
DATA_FOLDER+'/gcs_catalog.csv'